In [ ]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'SNP'

from ib_insync import util
util.startLoop()

# Set the root
from from_root import from_root
ROOT = from_root()

# set vscode path
import sys
sys.path.append(str(ROOT / 'src'))

# To import use 
# from src.<filename> import <function>
from src.utils import Vars, get_pickle

_vars = Vars(MARKET)
PORT = port = _vars.PORT

DATAPATH = ROOT / 'data' / MARKET

In [ ]:
df_chains = get_pickle(DATAPATH / 'df_chains.pkl')

In [26]:
from src.utils import get_closest_values
STDMULT = _vars.PUTSTDMULT
how_many = -2 # For Puts


In [43]:
def target_options_with_adjusted_sdev(df_chains: pd.DataFrame, 
                                      STDMULT: float, 
                                      how_many: int=-1) -> pd.DataFrame:
    
    """Adjust the standard deviation to DTE, penalizes DTES closer to zero"""

    # Get the extra SD adjusted to DTE
    xtra_sd = 1-(df_chains.dte/100)

    # Build the series for revised SD
    sd_revised = STDMULT + xtra_sd if STDMULT > 0 else STDMULT - xtra_sd
    # Identify the closest standerd devs to the revised SD

    df_ch = df_chains.assign(sd_revised=sd_revised)
    closest_sdevs = df_ch.groupby(['symbol', 'dte'])[['sdev', 'sd_revised']]\
        .apply(lambda x: get_closest_values(x.sdev, 
                                            x.sd_revised.min(), 
                                            how_many))
    closest_sdevs.name = 'sdev1' 

    # Join the closest chains to the closest revised SD
    df_ch1 = df_ch.set_index(['symbol', 'dte']).join(closest_sdevs)

    # Get the target chains
    df_ch2 = df_ch1[df_ch1.apply(lambda x: x.sdev in x.sdev1, axis=1)] \
                        .reset_index()
    
    return df_ch2

In [44]:
df_ch2

,symbol,dte,undId,expiry,strike,localSymbol,undPrice,iv,sigma,sdev,right,sd_revised,sdev1
0,AAL,2.705236,139673266,20240105,11.5,AAL,13.61,0.433205,0.507584,-4.156948,P,-2.972948,"[-3.1718894004967515, -4.1569482205267985]"
1,AAL,2.705236,139673266,20240105,12.0,AAL,13.61,0.433205,0.507584,-3.171889,P,-2.972948,"[-3.1718894004967515, -4.1569482205267985]"
2,AAL,9.705236,139673266,20240112,10.0,AAL,13.61,0.433205,0.961409,-3.754904,P,-2.902948,"[-3.234834528052406, -3.754904387867905]"
3,AAL,9.705236,139673266,20240112,10.5,AAL,13.61,0.433205,0.961409,-3.234835,P,-2.902948,"[-3.234834528052406, -3.754904387867905]"
4,AAL,16.705236,139673266,20240119,9.5,AAL,13.61,0.433205,1.261338,-3.258444,P,-2.832948,"[-2.862039506180123, -3.258443869916982]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2439,ZION,16.704264,276329,20240119,32.0,ZION,44.40,0.447337,4.248981,-2.918347,P,-2.832957,"[-2.9183467376736, -3.153697281034374]"
2440,ZION,23.704264,276329,20240126,29.0,ZION,44.40,0.447337,5.061560,-3.042540,P,-2.762957,"[-2.844972890278364, -3.0425404521032506]"
2441,ZION,23.704264,276329,20240126,30.0,ZION,44.40,0.447337,5.061560,-2.844973,P,-2.762957,"[-2.844972890278364, -3.0425404521032506]"
2442,ZION,30.704264,276329,20240202,27.5,ZION,44.40,0.447337,5.760636,-2.933704,P,-2.692957,"[-2.846907655419993, -2.933703620524261]"
